In [145]:
import psycopg2 # pip install psycopg2
import plotly.graph_objects as go # pip install plotly
import plotly.io as pio # save as image pip install kaleido
import pandas as pd

In [146]:
conn = psycopg2.connect(
    host="localhost",
    database="postgres",
    user="postgres",
    password="password",
)

In [147]:
def human_readable_format(number):
    number = float(number)
    if number >= 1e12:
        return f'{number / 1e12:.2f}T'  # Trillion
    elif number >= 1e9:
        return f'{number / 1e9:.2f}B'  # Billion
    elif number >= 1e6:
        return f'{number / 1e6:.2f}M'  # Million
    elif number >= 1e3:
        return f'{number / 1e3:.2f}K'  # Thousand
    else:
        return str(number)  # Less than thousand, return as is

In [148]:
# regular stock chart interactive

cursor = conn.cursor()
query = """
SELECT 
    info->>'symbol' AS symbol,
    info->>'shortName' AS short_name,
    array_agg(
        json_build_object(
            'close_date', to_timestamp((key)::numeric / 1000), 
            'price', value::numeric
        )
    ) AS close_history
FROM stocks.yfinance y,
    jsonb_each_text(weekly->'Close') AS close_data(key, value)
WHERE ticker = 'AAPL'
GROUP BY symbol, short_name;
"""
cursor.execute(query)
result = cursor.fetchone()
cursor.close()

close_history = result[2]
dates = []
prices = []

for entry in close_history:
    dates.append(entry['close_date'])
    prices.append(entry['price'])

fig = go.Figure(data=go.Scatter(x=dates, y=prices, mode='lines+markers'))

fig.update_layout(
    title="Stock Close Price Over Time (AAPL)",
    xaxis_title="Date",
    yaxis_title="Close Price (USD)",
    xaxis=dict(tickformat="%Y-%m-%d", tickangle=45),
    template="plotly_dark"
)

fig.show()

In [149]:
# fig.write_html("interactive_plot.html", include_plotlyjs='cdn', full_html=False)
# fig.to_html(include_plotlyjs='cdn', full_html=False)
# fig.write_image("stock_close_price.png")

In [150]:
# Market Cap by Sector

cursor = conn.cursor()
query = """
SELECT 
    info->>'symbol' AS symbol,
    info->>'shortName' AS short_name,
    info->>'sector' AS sector,
    (info->>'marketCap')::numeric AS market_cap
FROM stocks.yfinance y;
"""
cursor.execute(query)

data = cursor.fetchall()
cursor.close()

df = pd.DataFrame(data, columns=['symbol', 'short_name', 'sector', 'market_cap'])

sector_market_cap = df.groupby('sector')['market_cap'].sum().reset_index()

fig = go.Figure(go.Pie(
    labels=sector_market_cap['sector'],
    values=sector_market_cap['market_cap'],
    hole=0.3,
    textinfo='none',
    hoverinfo='label+text+percent',
    text=sector_market_cap['market_cap'].apply(human_readable_format),
))

fig.update_layout(
    title="Market Cap by Sector",
    template="plotly_dark",
)

fig.show()

In [151]:
# Top 20 Companies by Market Cap in the Technology Sector

sector = 'Technology'

cursor = conn.cursor()
query = f"""
SELECT 
    info->>'symbol' AS symbol,
    info->>'shortName' AS short_name,
    info->>'sector' AS sector,
    (info->>'marketCap')::numeric AS market_cap
FROM stocks.yfinance y
WHERE info->>'sector' = '{sector}'
ORDER BY market_cap DESC
LIMIT 20;
"""
cursor.execute(query)
data = cursor.fetchall()

df = pd.DataFrame(data, columns=['symbol', 'short_name', 'sector', 'market_cap'])

#df = df[~df['symbol'].isin(['AAPL', 'MSFT', 'NVDA'])]
df['formatted_market_cap'] = df['market_cap'].apply(human_readable_format)

fig = go.Figure(go.Pie(
    labels=df['short_name'],
    values=df['market_cap'], 
    hole=0.3,
    textinfo='none',
    hoverinfo='label+text+percent',
    text=df['formatted_market_cap'],
))

fig.update_layout(
    title=f"Top 20 Companies by Market Cap in the {sector} Sector",
    template="plotly_dark",
)

fig.show()

In [152]:
# Add a few stocks to a single chart

time_frame = 'daily'
stocks = ",".join(['AAPL', 'MSFT', 'NVDA'])
quoted_stocks = ", ".join([f"'{stock}'" for stock in ['AAPL', 'MSFT', 'NVDA']])

cursor = conn.cursor()
query = f"""
SELECT
    info->>'symbol' AS symbol,
    info->>'shortName' AS short_name,
    array_agg(
        json_build_object(
            'close_date', to_timestamp((key)::numeric / 1000), 
            'price', value::numeric
        )
    ) AS close_history
FROM stocks.yfinance y,
    jsonb_each_text({time_frame}->'Close') AS close_data(key, value)
WHERE ticker IN ({quoted_stocks})
GROUP BY symbol, short_name;
"""
cursor.execute(query)
data = cursor.fetchall()
cursor.close()

fig = go.Figure()

for row in data:
    symbol = row[0]
    short_name = row[1]
    close_history = row[2]
    
    dates = []
    prices = []

    for entry in close_history:
        dates.append(entry['close_date'])
        prices.append(entry['price'])

    fig.add_trace(go.Scatter(x=dates, y=prices, mode='lines+markers', name=symbol))

fig.update_layout(
    title="Stock Close Price Over Time",
    xaxis_title="Date",
    yaxis_title="Close Price (USD)",
    xaxis=dict(tickformat="%Y-%m-%d", tickangle=45),
    template="plotly_dark"
)

fig.show()

In [153]:
# mini chart for table

cursor = conn.cursor()
query = """
    SELECT 
        info->>'symbol' AS symbol,
        info->>'shortName' AS short_name,
        array_agg(
            json_build_object(
                'close_date', to_timestamp((key)::numeric / 1000), 
                'price', value::numeric
            )
        ) AS close_history
    FROM stocks.yfinance y,
        jsonb_each_text(hourly->'Close') AS close_data(key, value)
    WHERE ticker = 'AAPL'
    GROUP BY symbol, short_name;
"""
cursor.execute(query)
result = cursor.fetchone()
cursor.close()

close_history = result[2]
dates = []
prices = []

for entry in close_history:
    dates.append(entry['close_date'])
    prices.append(entry['price'])

line_color = 'green' if prices[-1] >= prices[0] else 'red'

fig = go.Figure(data=go.Scatter(
    x=dates, 
    y=prices,
    line=dict(color=line_color, width=6),
))

fig.update_layout(
    xaxis=dict(
        showticklabels=False,
        showgrid=False,
    ),
    yaxis=dict(
        showticklabels=False,
        showgrid=False,
    ),
    template="plotly_dark"
)

fig.show()